In [63]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import sqlite3
import seaborn as sns
from scipy import sparse
import requests
import json
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'seaborn'

In [55]:
WOS_DISAMBIGUATION_DB = "../data/wos-disambiguation-data.db"

In [56]:
conn = sqlite3.connect(WOS_DISAMBIGUATION_DB)
cur = conn.cursor()

In [58]:
name_table = pd.read_sql("select * from name_table", conn)
name_paper_table = pd.read_sql("select * from name_paper_table", conn)

In [60]:
display(name_table.head())
display(name_paper_table.head())

,name,initials,first_name,last_name,normalized_name,block_id,name_id
0,"TORRES, JM",JT,JM,TORRES,torresj,0,0
1,"SANCHEZ, C",CS,C,SANCHEZ,sanchezc,1,1
2,"SUNE, C",CS,C,SUNE,sunec,2,2
3,"SMERDOU, C",CS,C,SMERDOU,smerdouc,3,3
4,"PREVEC, L",LP,L,PREVEC,prevecl,4,4


,name_id,paper_id,email_address
0,0,WOS:A1995TE73200023,None
1,1,WOS:A1995TE73200023,None
2,2,WOS:A1995TE73200023,None
3,3,WOS:A1995TE73200023,None
4,4,WOS:A1995TE73200023,None


In [61]:
name_paper_table = pd.merge(name_paper_table, name_table, on="name_id", how="left")
name_paper_table.head()

,name_id,paper_id,email_address,name,initials,first_name,last_name,normalized_name,block_id
0,0,WOS:A1995TE73200023,None,"TORRES, JM",JT,JM,TORRES,torresj,0
1,1,WOS:A1995TE73200023,None,"SANCHEZ, C",CS,C,SANCHEZ,sanchezc,1
2,2,WOS:A1995TE73200023,None,"SUNE, C",CS,C,SUNE,sunec,2
3,3,WOS:A1995TE73200023,None,"SMERDOU, C",CS,C,SMERDOU,smerdouc,3
4,4,WOS:A1995TE73200023,None,"PREVEC, L",LP,L,PREVEC,prevecl,4


In [96]:
OUTPUT_DIR = "../data/name-blocks"

for block_id, df in name_paper_table.groupby("block_id"):
    papers = df[["paper_id"]]

    paper_file_name = OUTPUT_DIR + "/papers-%d.csv" % block_id
    if os.path.exists(paper_file_name):
        continue
    else:
        papers.assign(block_id=block_id).to_csv(paper_file_name, index=False)